In [ ]:
# env = gym.make("CarRacing-v0")
# observation = env.reset()
# for _ in range(5):
#     snap = image.fromarray(observation)
#     snap.show()
#     env.render()
#     action = env.action_space.sample() 
#     observation, reward, done, info = env.step(action)    
#     print(reward)   
#     print(np.shape(observation))
#     if done: 
#       break;
# env.close()

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as image
import pyvirtualdisplay

from tqdm import tqdm
import tensorflow as tf
import gym
# from tf_agents.agents.PPOagent import PPOagent
from tf_agents.agents.ppo import ppo_clip_agent
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import value_network
from tf_agents.networks import actor_distribution_network
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.specs import array_spec
import tf_agents.trajectories.time_step as ts
from tf_agents.environments import py_environment, utils
from tf_agents.policies import greedy_policy
from tf_agents.policies import random_tf_policy

In [2]:

class LunarEvniornment(py_environment.PyEnvironment):

    def __init__(self):
        super().__init__()
        self._action_spec = array_spec.BoundedArraySpec(shape=(2,), dtype=np.float32, name='action', minimum=-1, maximum=1.0)
        self._observation_spec = array_spec.BoundedArraySpec(shape=(8,), dtype=np.float32, name='observation', minimum=-3.4028234663852886e+38, maximum=3.4028234663852886e+38)


        self.done = False
        print("init")
        self.game = gym.make('LunarLanderContinuous-v2')
        # self.game = gym.make('CartPole-v0')
        self.game.reset()
        self.game.render()
        # self.game.reset()

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        state = self.game.reset() #makesure np array
        state = self.processimg(state)
        return ts.restart(state)

    def _step(self, action):

        # if self.done:
        #     # The last action ended the episode. Ignore the current action and start a new episode.
        #     return self.reset()
        nextState, reward, done, info = self.game.step(action)
        self.done = done
        nextState = self.processimg(nextState)
        
        if self.done:
            return ts.termination(nextState, reward)
        else:
            return ts.transition(nextState, reward)

    def render(self, mode='rgb_array'):
        return self.game.render()
    
    def processimg(self, state):
        # state = state[0:-50] # if need to crop
        # snap = image.fromarray(state).convert('LA') #Grayscale if need to optimize

        # snap = image.fromarray(state)
        # snap.show()

        return state

    
class CarEvniornment(py_environment.PyEnvironment):

    def __init__(self):
        super().__init__()
        self._action_spec = array_spec.BoundedArraySpec(shape=(3,), dtype=np.float32, name='action', minimum=[-1,  0,  0], maximum=1.0)
        self._observation_spec = array_spec.BoundedArraySpec(shape=(96, 96, 3), dtype=np.float32, name='observation', minimum=0, maximum=1)


        self.done = False
        print("init")
        self.game = gym.make('CarRacing-v0')
        # self.game = gym.make('CartPole-v0')
        self.game.reset()
        self.game.render()
        # self.game.reset()

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        state = self.game.reset() #makesure np array
        state = self.processimg(state)
        return ts.restart(state)

    def _step(self, action):

        # if self.done:
        #     # The last action ended the episode. Ignore the current action and start a new episode.
        #     return self.reset()
        nextState, reward, done, info = self.game.step(action)
        self.done = done
        nextState = self.processimg(nextState)
        
        if self.done:
            return ts.termination(nextState, reward)
        else:
            return ts.transition(nextState, reward)

    def render(self, mode='rgb_array'):
        return self.game.render()
    
    def processimg(self, state):
        # state = state[0:-50] # if need to crop
        # snap = image.fromarray(state).convert('LA') #Grayscale if need to optimize

        # snap = image.fromarray(state)
        # snap.show()

        return np.divide(state, 255, dtype=np.float32)


class CartPoleEnv(py_environment.PyEnvironment):

    def __init__(self):
        super().__init__()


        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int64, name='action', minimum=0, maximum = 1)
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(4,), 
            dtype=np.float64, name='observation', minimum=np.array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38], 
            dtype=np.float64), maximum=np.array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],dtype=np.float64)
            )


        self.done = False
        print("init")
        # self.game = gym.make('CarRacing-v0')
        self.game = gym.make('CartPole-v0')
        self.game.reset()
        self.game.render()
        # self.game.reset()

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        state = self.game.reset() #makesure np array
        return ts.restart(state)

    def _step(self, action):

        # if self.done:
        #     # The last action ended the episode. Ignore the current action and start a new episode.
        #     return self.reset()
        nextState, reward, done, info = self.game.step(action)
        self.done = done
        
        if self.done:
            return ts.termination(nextState, reward)
        else:
            return ts.transition(nextState, reward)

    def render(self, mode='rgb_array'):
        return self.game.render()
    



# if __name__ == "__main__":
# 	environment = CarEvniornment()
# 	utils.validate_py_environment(environment, episodes=5)

In [3]:
num_iterations = 500 # @param {type:"integer"}
collect_episodes_per_iteration = 2 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

fc_layer_params = (100,)

learning_rate = 1e-3 # @param {type:"number"}
log_interval = 25 # @param {type:"integer"}
num_eval_episodes = 5 # @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}

In [4]:
env = LunarEvniornment()

train_env = tf_py_environment.TFPyEnvironment(env)

# env.reset()
# state = env.render()
# print(env.observation_spec())
# for _ in range(10):
#     env.step([0,0,0])

# state = env.render()
# state = state[0:-50]
# # snap = image.fromarray(state).convert('LA') #Grayscale if need to optimize
# snap = image.fromarray(state)
# snap = snap.resize((480,275))
# snap.show()
# np.shape(snap)

init


In [5]:
# env2 = suite_gym.load('CartPole-v0')
# env3 = tf_py_environment.TFPyEnvironment(env2)

# print(env2.observation_spec())
# print(env2.time_step_spec())
# print(env2.action_spec())
# print('\n')
# print(train_env.observation_spec())
# print(train_env.time_step_spec())
# print(train_env.action_spec())

In [6]:
optim = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
clip = 0.2
counter = tf.compat.v1.train.get_or_create_global_step() #how many times the variable was updated


actor_net = actor_distribution_network.ActorDistributionNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=(256,256,),
    # conv_layer_params=[(16, 8, 4), (32, 4, 2)],
)
value_net = value_network.ValueNetwork(
    train_env.observation_spec(),
    fc_layer_params=(256,256,),
    # conv_layer_params=[(16, 8, 4), (32, 4, 2)],
)

agent = ppo_clip_agent.PPOClipAgent(
    train_env.time_step_spec(), 
    train_env.action_spec(), 
    optimizer=optim, 
    actor_net=actor_net, 
    value_net=value_net,
    num_epochs=25,
    train_step_counter=counter,
    discount_factor=0.99,
    gradient_clipping=0.5,
    entropy_regularization=1e-2,
    importance_ratio_clipping=0.2, #from the paper
    use_gae=True,
    use_td_lambda_return=True
)

agent.initialize()

In [7]:
# policy
# experience buffer
# main loop

In [8]:
def compute_avg_return(environment, policy, num_episodes=5):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

In [9]:
eval_policy = greedy_policy.GreedyPolicy(agent.policy)
# eval_policy = tf_agent.policy
collect_policy = agent.collect_policy

In [10]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=1,
    max_length=10000)

In [11]:
agent.collect_data_spec

_TupleWrapper(Trajectory(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), observation=BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32)), action=BoundedTensorSpec(shape=(2,), dtype=tf.float32, name='action', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)), policy_info=DictWrapper({'loc': TensorSpec(shape=(2,), dtype=tf.float32, name='NormalProjectionNetwork_loc'), 'scale': TensorSpec(shape=(2,), dtype=tf.float32, name='NormalProjectionNetwork_scale')}), next_step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32))))

In [12]:
collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
    train_env, 
    collect_policy, 
    observers=[replay_buffer.add_batch],
    num_episodes=64
    )


In [13]:
def test():
    for _ in range(2):
        time_step = train_env.reset()
        env.game.render()#np.reshape(time_step[-1]*255, (96,96,3))
        counter = 0
        while not time_step.is_last():
            action_step = agent.policy.action(time_step)
            time_step = train_env.step(action_step.action)
            counter += 1
            env.game.render()
        print(counter)


In [15]:
def compute_avg_return(environment, policy, num_episodes=5):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
    avg_return = compute_avg_return(train_env, eval_policy, 1)
        print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns.append(avg_return)

IndentationError: unexpected indent (<ipython-input-15-b3dd9d8a500d>, line 16)

In [20]:
returns = []
for i in tqdm(range(100)):
    collect_driver.run(maximum_iterations = 300)
    # print("collected")
    experiences = replay_buffer.gather_all()
    loss, _ = agent.train(experience= experiences)
    print(loss)
    if i % 5 == 0:
        avg_return = compute_avg_return(train_env, eval_policy, 1)
        print('step = {0}: Average Return = {1}'.format(i, avg_return))
        returns.append(avg_return)
    # test()
    # print("Loss:", loss)
    # print("\n")
    replay_buffer.clear()
    # avg_return = compute_avg_return(train_env, eval_policy, 1)
    # print("avg return:", avg_return)
    # print("\n")


  5%|▌         | 5/100 [00:12<03:58,  2.51s/it]tf.Tensor(581.0404, shape=(), dtype=float32)
tf.Tensor(595.00146, shape=(), dtype=float32)
 10%|█         | 10/100 [00:27<04:21,  2.91s/it]tf.Tensor(540.8396, shape=(), dtype=float32)
tf.Tensor(489.75916, shape=(), dtype=float32)
 15%|█▌        | 15/100 [00:44<04:39,  3.29s/it]tf.Tensor(340.17627, shape=(), dtype=float32)
tf.Tensor(315.23233, shape=(), dtype=float32)
 20%|██        | 20/100 [01:03<04:59,  3.75s/it]tf.Tensor(303.23898, shape=(), dtype=float32)
tf.Tensor(266.2768, shape=(), dtype=float32)
 25%|██▌       | 25/100 [01:28<05:50,  4.67s/it]tf.Tensor(217.82759, shape=(), dtype=float32)
tf.Tensor(220.54106, shape=(), dtype=float32)
 30%|███       | 30/100 [01:54<05:51,  5.02s/it]tf.Tensor(206.6404, shape=(), dtype=float32)
tf.Tensor(227.8856, shape=(), dtype=float32)
 35%|███▌      | 35/100 [02:24<06:18,  5.82s/it]tf.Tensor(202.7655, shape=(), dtype=float32)
tf.Tensor(200.67528, shape=(), dtype=float32)
 36%|███▌      | 36/100 [02

KeyboardInterrupt: 

In [ ]:
test()

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" contr/ols>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

In [ ]:
num_episodes = 2
video_filename = 'cartpole.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = train_env.reset()
    data = np.reshape(time_step[-1]*255, (96,96,3))
    video.append_data(data)
    while not time_step.is_last():
      action_step = agent.policy.action(time_step)
      time_step = train_env.step(action_step.action)
      data = np.reshape(time_step[-1]*255, (96,96,3))
      video.append_data(data)

embed_mp4(video_filename)